In [57]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [58]:
data = pd.read_csv('/content/arithmatic_dataset.csv')

In [59]:
data.head()

,Message,Intent
0,What is 5 + 3?,addition
1,Calculate 15 - 7,subtraction
2,Please solve 6 * 4,multiplication
3,Divide 20 by 5,division
4,What's the result of 12 + 9?,addition


## ***EDA***

In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 332 entries, 0 to 331
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Message  332 non-null    object
 1   Intent   332 non-null    object
dtypes: object(2)
memory usage: 5.3+ KB


In [61]:
data.duplicated().value_counts()

False    226
True     106
dtype: int64

In [62]:
data[data.duplicated(keep=False)==True]

,Message,Intent
6,Multiply 7 and 6,multiplication
11,Divide 42 by 7,division
19,Find the value of 48 / 6,division
23,Divide 63 by 9,division
39,Divide 36 by 6,division
...,...,...
327,Divide 8 by 8,division
328,Calculate 32 + 2,addition
329,12 minus 1,subtraction
330,Multiply 7 and 1,multiplication


In [63]:
data.drop_duplicates(inplace=True)

In [64]:
data.duplicated().value_counts()

False    226
dtype: int64

## ***Model***

In [65]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [66]:
# Extract messages and intents
train_messages = train_data['Message']
train_intents = train_data['Intent']
test_messages = test_data['Message']
test_intents = test_data['Intent']

In [67]:
# Tokenize messages
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train_messages)

In [68]:
train_sequences = tokenizer.texts_to_sequences(train_messages)
test_sequences = tokenizer.texts_to_sequences(test_messages)

In [69]:
vocab_size = len(tokenizer.word_index) + 1

In [70]:
# Encode intents using LabelEncoder
encoder = LabelEncoder()
train_labels = encoder.fit_transform(train_intents)
test_labels = encoder.transform(test_intents)

In [71]:
# Pad sequences
max_seq_length = max(map(len, train_sequences))
train_sequences = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_seq_length)
test_sequences = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_seq_length)

In [72]:
# Build the model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=64, input_length=max_seq_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(encoder.classes_), activation='softmax')
])

In [73]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 6, 64)             4608      
                                                                 
 global_average_pooling1d_2   (None, 64)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 4)                 260       
                                                                 
Total params: 9,028
Trainable params: 9,028
Non-trainable params: 0
_________________________________________________________________


In [74]:
# Train the model
model.fit(train_sequences, train_labels, epochs=50, batch_size=32, validation_split=0.1)

Epoch 1/50
6/6 [==============================] - 2s 185ms/step - loss: 1.3831 - accuracy: 0.2778 - val_loss: 1.3692 - val_accuracy: 0.3333
Epoch 2/50
6/6 [==============================] - 1s 101ms/step - loss: 1.3627 - accuracy: 0.3457 - val_loss: 1.3501 - val_accuracy: 0.3333
Epoch 3/50
6/6 [==============================] - 0s 42ms/step - loss: 1.3423 - accuracy: 0.3704 - val_loss: 1.3282 - val_accuracy: 0.3333
Epoch 4/50
6/6 [==============================] - 0s 42ms/step - loss: 1.3185 - accuracy: 0.4136 - val_loss: 1.3027 - val_accuracy: 0.3333
Epoch 5/50
6/6 [==============================] - 0s 14ms/step - loss: 1.2906 - accuracy: 0.4444 - val_loss: 1.2725 - val_accuracy: 0.3333
Epoch 6/50
6/6 [==============================] - 0s 61ms/step - loss: 1.2584 - accuracy: 0.5000 - val_loss: 1.2376 - val_accuracy: 0.5000
Epoch 7/50
6/6 [==============================] - 0s 69ms/step - loss: 1.2224 - accuracy: 0.5370 - val_loss: 1.1964 - val_accuracy: 0.5000
Epoch 8/50
6/6 [=========

In [75]:
# Evaluate the model
loss, accuracy = model.evaluate(test_sequences, test_labels)
print(f"Test accuracy: {accuracy:.4f}")

2/2 [==============================] - 0s 8ms/step - loss: 0.1516 - accuracy: 0.9348
Test accuracy: 0.9348


In [76]:
# Make predictions
predicted_probabilities = model.predict(test_sequences)
predicted_labels = predicted_probabilities.argmax(axis=1)
class_names = encoder.classes_
classification_rep = classification_report(test_labels, predicted_labels, target_names=class_names)
print(classification_rep)

2/2 [==============================] - 0s 4ms/step
                precision    recall  f1-score   support

      addition       0.80      1.00      0.89        12
      division       1.00      1.00      1.00        12
multiplication       1.00      0.80      0.89        10
   subtraction       1.00      0.92      0.96        12

      accuracy                           0.93        46
     macro avg       0.95      0.93      0.93        46
  weighted avg       0.95      0.93      0.94        46



In [77]:
# After training the model
model.save('arthimatic_model.h5')

# ***Test Model***

In [78]:
import tensorflow as tf

def test_intent_recognition_model(model_filename, tokenizer, encoder):
    # Load the saved model
    loaded_model = tf.keras.models.load_model(model_filename)

    while True:
        # Get user input
        user_input = input("Enter an arithmetic problem message (or 'exit' to quit): ")
        if user_input.lower() in ['exit', 'quit']:
            print("Exiting...")
            break

        # Tokenize and pad user input
        user_sequence = tokenizer.texts_to_sequences([user_input])
        user_sequence = tf.keras.preprocessing.sequence.pad_sequences(user_sequence, maxlen=max_seq_length)

        # Predict intent
        predicted_probability = loaded_model.predict(user_sequence)
        predicted_label = predicted_probability.argmax()
        predicted_intent = encoder.inverse_transform([predicted_label])[0]

        print(f"Predicted Intent: {predicted_intent}")

if __name__ == "__main__":
    # Load tokenizer and encoder
    tokenizer = tf.keras.preprocessing.text.Tokenizer()
    tokenizer.fit_on_texts(train_messages)
    max_seq_length = max(map(len, train_sequences))

    encoder = LabelEncoder()
    train_labels = encoder.fit_transform(train_intents)

    # Load the trained model in .h5 format and test using the function
    model_filename = '/content/arthimatic_model.h5'
    test_intent_recognition_model(model_filename, tokenizer, encoder)

Enter an arithmetic problem message (or 'exit' to quit): exit
Exiting...


# ***Testing Code***

In [96]:
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')
# Load the pre-trained model
model = tf.keras.models.load_model('/content/arthimatic_model.h5')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [97]:
math_keywords = {
    'add': {'plus', 'add', '+', 'sum'},
    'subtract': {'minus', 'subtract', '-', 'difference'},
    'multiply': {'times', 'multiply', '*', 'product'},
    'divide': {'divide', '/', 'over'}
}

In [98]:
def perform_math_operation(operation, operand1, operand2):
    if operation == 'add':
        return operand1 + operand2
    elif operation == 'subtract':
        return operand1 - operand2
    elif operation == 'multiply':
        return operand1 * operand2
    elif operation == 'divide':
        if operand2 == 0:
            return "Cannot divide by zero"
        return operand1 / operand2

In [99]:
def process_math_question(question):
    tokens = word_tokenize(question.lower())

    math_action = None
    operand1 = None
    operand2 = None

    for token in tokens:
        for operation, keywords in math_keywords.items():
            if token in keywords:
                math_action = operation
                break
        try:
            token_value = float(token)
            if operand1 is None:
                operand1 = token_value
            else:
                operand2 = token_value
        except ValueError:
            pass

    if math_action and operand1 is not None and operand2 is not None:
        result = perform_math_operation(math_action, operand1, operand2)
        return f"The result of {operand1} {math_action} {operand2} is {result}"
    elif operand1 is not None and operand2 is None:
        return f"Please specify the operation for {operand1}"
    else:
        return "Sorry, I couldn't understand the question."

In [100]:
while True:
    user_question = input("Enter a math-related question (or 'quit' to exit): ")
    if user_question.lower() == "quit":
        break

    response = process_math_question(user_question.lower())
    print(response)

Enter a math-related question (or 'quit' to exit): Subtract 3.5 from 7.8
The result of 3.5 subtract 7.8 is -4.3
Enter a math-related question (or 'quit' to exit): Calculate 5 * 2"
The result of 5.0 multiply 2.0 is 10.0
Enter a math-related question (or 'quit' to exit): quit
